# Лаба 6. Суперачивка. Построение неперсонализированной рекомендательной системы для фильмов

### По имеющимся данным о рейтингах фильмов (MovieLens: 100 000 рейтингов) посчитать агрегированную статистику по ним.

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
import json

## Константы из ЛК

In [339]:
k = 10
z = 3.291

## Загрузим файл

In [340]:
df = pd.read_csv(
    'u.data', 
    delimiter='\t', 
    header=None, 
    names=['user_id', 'item_id','rating','timestamp']
)

In [421]:
df

,user_id,item_id,rating,timestamp,positive
0,196,242,3,881250949,0
1,186,302,3,891717742,0
2,22,377,1,878887116,0
3,244,51,2,880606923,0
4,166,346,1,886397596,0
...,...,...,...,...,...
99995,880,476,3,880175444,0
99996,716,204,5,879795543,1
99997,276,1090,1,874795795,0
99998,13,225,2,882399156,0


## Добавим неагрегированные атрибуты

In [342]:
df['positive'] = np.where(df.rating >= 4,1,0)

## Глобальное среднее

In [343]:
global_avg = df.rating.mean()
global_avg

3.52986

## Добавим агрегаты

1. Количество человек n, поставивших рейтинг фильму .     
2. Средний рейтинг фильма (сумма рейтингов фильма / количество человек, оценивших фильм)     
3. Количество человек m, оценивших фильм положительно. Оценки 4 и выше считаются положительными.
4. Доля людей, оценивших фильм положительно (пункт 3 / пункт 1)    
5. Глобальное среднее по всему датасету. Сумма всех оценок по всем фильмам /Количество всех оценок по всем фильмам

In [504]:
df_aggr = df.groupby('item_id').agg(
    n = ('user_id', 'count'),
    rating_sum = ('rating', 'sum'),
    rating_avg = ('rating', 'mean'),
    m = ('positive', 'sum')
)

In [505]:
df_aggr['p'] = df_aggr.m / df_aggr.n

In [506]:
# Вернем id в столбцы чтобы не ошибиться с индексами
df_aggr = \
    df_aggr\
    .reset_index(col_fill='item_id')\
    .rename(columns={'index':'item_id'})

In [507]:
df_aggr

,item_id,n,rating_sum,rating_avg,m,p
0,1,452,1753,3.878319,321,0.710177
1,2,131,420,3.206107,51,0.389313
2,3,90,273,3.033333,34,0.377778
3,4,209,742,3.550239,122,0.583732
4,5,86,284,3.302326,39,0.453488
...,...,...,...,...,...,...
1677,1678,1,1,1.000000,0,0.000000
1678,1679,1,3,3.000000,0,0.000000
1679,1680,1,2,2.000000,0,0.000000
1680,1681,1,3,3.000000,0,0.000000


## Вычисления поверх агрегатов

In [508]:
#Оценку, поправленную на нехватку данных
def avg_adj(df):
    result = (df.rating_sum + k*global_avg) / (df.n + k)
    return result

In [510]:
#Нижнюю и верхнюю границы доверительного интервала оценки
#(Wilson score interval) из лекции с заданным уровнем доверия
def wilson(df):
    denominator = 1 + z*z/df.n
    centre_adj_prob = df.p + z*z / (2*df.n)
    adj_std_deviation = sqrt((df.p*(1 - df.p))/ df.n + z*z / (4*df.n**2)) 
    
    lower_bound = (centre_adj_prob - z*adj_std_deviation) / denominator
    upper_bound = (centre_adj_prob + z*adj_std_deviation) / denominator
    return lower_bound, upper_bound

In [511]:
df_aggr['avg_adj'] = df_aggr.apply(avg_adj, axis = 1)

In [512]:
# Если функция выдает больше 1 столбца то сначала проще распаковать в df
df_wilson = \
    pd.DataFrame(df_aggr.apply(wilson, axis = 1).tolist())\
    .rename(columns={0:'lower_bound', 1:'upper_bound'})

In [513]:
# добавить доверительный интервал к основному df
df_aggr = \
    df_aggr.merge(
        df_wilson, 
        left_index=True, 
        right_index=True, 
        how='left'
)

In [514]:
df_aggr

,item_id,n,rating_sum,rating_avg,m,p,avg_adj,lower_bound,upper_bound
0,1,452,1753,3.878319,321,0.710177,3.870776,0.635683,0.774834
1,2,131,420,3.206107,51,0.389313,3.229068,0.262759,0.532772
2,3,90,273,3.033333,34,0.377778,3.082986,0.231465,0.550347
3,4,209,742,3.550239,122,0.583732,3.549309,0.470114,0.689099
4,5,86,284,3.302326,39,0.453488,3.326027,0.292113,0.625268
...,...,...,...,...,...,...,...,...,...
1677,1678,1,1,1.000000,0,0.000000,3.299873,0.000000,0.915474
1678,1679,1,3,3.000000,0,0.000000,3.481691,0.000000,0.915474
1679,1680,1,2,2.000000,0,0.000000,3.390782,0.000000,0.915474
1680,1681,1,3,3.000000,0,0.000000,3.481691,0.000000,0.915474


## Добавим названия фильмов

Были сложности при копировании файла из-за кодировки
```bash
hadoop fs -cat /labs/lab06data/ml-100k/u.item | iconv -c -t UTF-8 > u.item
```


In [515]:
item_col_lst = [
    
    'movie_id',
    'movie_title',
    'release_date',
    'video_release_date',
    'imdb_url',
    'unknown',
    'action',
    'adventure',
    'animation',
    'children',
    'comedy',
    'crime',
    'documentary',
    'drama',
    'fantasy',
    'film-Noir', 
    'horror',
    'musical',
    'mystery',
    'romance',
    'sci-Fi',
    'thriller',
    'war',
    'western',

]

In [516]:
df_item_ref = pd.read_csv(
    'u.item', 
    delimiter='|', 
    header=None, 
    names=item_col_lst,
    index_col=None
)

In [517]:
df_item_ref.head(3)

,movie_id,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children,...,fantasy,film-Noir,horror,musical,mystery,romance,sci-Fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [518]:
df_aggr = \
    df_aggr.merge(
        df_item_ref[['movie_id','movie_title']],
        left_on='item_id',
        right_on='movie_id'
)

df_aggr

,item_id,n,rating_sum,rating_avg,m,p,avg_adj,lower_bound,upper_bound,movie_id,movie_title
0,1,452,1753,3.878319,321,0.710177,3.870776,0.635683,0.774834,1,Toy Story (1995)
1,2,131,420,3.206107,51,0.389313,3.229068,0.262759,0.532772,2,GoldenEye (1995)
2,3,90,273,3.033333,34,0.377778,3.082986,0.231465,0.550347,3,Four Rooms (1995)
3,4,209,742,3.550239,122,0.583732,3.549309,0.470114,0.689099,4,Get Shorty (1995)
4,5,86,284,3.302326,39,0.453488,3.326027,0.292113,0.625268,5,Copycat (1995)
...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,1,1,1.000000,0,0.000000,3.299873,0.000000,0.915474,1678,Mat' i syn (1997)
1678,1679,1,3,3.000000,0,0.000000,3.481691,0.000000,0.915474,1679,B. Monkey (1998)
1679,1680,1,2,2.000000,0,0.000000,3.390782,0.000000,0.915474,1680,Sliding Doors (1998)
1680,1681,1,3,3.000000,0,0.000000,3.481691,0.000000,0.915474,1681,You So Crazy (1994)


## Рейтинги

In [525]:
def get_top10_ids(field):
    result = \
        df_aggr\
        .sort_values(by=[field, 'movie_title'], ascending=(False, True))\
        .head(10)['item_id']\
        .tolist()
    print(result)
    return result

In [528]:
top10_rates = get_top10_ids('n')
top10_average = get_top10_ids('rating_avg')
top10_rating = get_top10_ids('avg_adj')
top10_lower = get_top10_ids('lower_bound')

[50, 258, 100, 181, 294, 286, 288, 1, 300, 121]
[1536, 1653, 814, 1201, 1189, 1467, 1500, 1599, 1293, 1122]
[318, 483, 64, 408, 169, 12, 603, 50, 114, 178]
[64, 98, 318, 479, 50, 483, 603, 427, 357, 12]


## Результат

In [523]:
result_dic = {
    "top10_rates": top10_rates,
    "top10_average": top10_average,
    "top10_rating": top10_rating,
    "top10_lower": top10_lower 
}

In [524]:
with open('../lab06s.json', 'w') as outfile:
    json.dump(result_dic, outfile)